# Projeto: treinamento da Rede Yolo com dataset Coco128

## 1 Montar as pastas e criar o arquivo yaml

In [ ]:
# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Definir caminho da pasta coco128 no Google Drive
import os

base_path = "/content/drive/MyDrive/coco128"
os.makedirs(base_path, exist_ok=True)

# Criar coco128.yaml
yaml_content = """path: /content/drive/MyDrive/coco128

train: images/train2017
val: images/train2017  # aqui usamos o mesmo conjunto como validação

names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: broccoli
  51: carrot
  52: hot dog
  53: pizza
  54: donut
  55: cake
  56: chair
  57: couch
  58: potted plant
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: laptop
  64: mouse
  65: remote
  66: keyboard
  67: cell phone
  68: microwave
  69: oven
  70: toaster
  71: sink
  72: refrigerator
  73: book
  74: clock
  75: vase
  76: scissors
  77: teddy bear
  78: hair drier
  79: toothbrush
"""

yaml_path = os.path.join(base_path, "coco128.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"✅ Arquivo coco128.yaml criado em: {yaml_path}")

## 2 Instalar YOLOv8

In [14]:
# Instalar YOLOv8 (Ultralytics)
!pip install ultralytics

## 3 Importar biblioteca YOLO e definir caminho base

In [15]:
# Importar biblioteca YOLO e definir caminho base
from ultralytics import YOLO
import os, shutil

In [16]:
base_path = "/content/drive/MyDrive/coco128"
runs_path = os.path.join(base_path, "runs")
os.makedirs(runs_path, exist_ok=True)

print("📂 Dataset localizado em:", base_path)
print("📂 Resultados serão salvos em:", runs_path)

📂 Dataset localizado em: /content/drive/MyDrive/coco128
📂 Resultados serão salvos em: /content/drive/MyDrive/coco128/runs


## 4 Carregar modelo pré-treinado

In [17]:
# Carregar modelo pré-treinado (nano = mais leve/rápido)
model = YOLO("yolov8n.pt")

## 5 Treinar modelo no coco128

In [27]:
# Treinar modelo no coco128 (imagens + labels do Drive)
results = model.train(
    data=f"{base_path}/coco128.yaml",  # dataset.yaml no Drive
    epochs=10,                         # número de épocas
    imgsz=640,                         # tamanho das imagens
    batch=16,                          # batch size
    project=runs_path,                 # onde salvar
    name="exp_coco128"                 # nome da pasta do experimento
)


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/coco128/coco128.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_coco128, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

## 6 Avaliar modelo no conjunto de validação

In [28]:
# Avaliar modelo no conjunto de validação
metrics = model.val()


Ultralytics 8.3.189 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.4±0.2 ms, read: 26.5±9.2 MB/s, size: 53.4 KB)
val: Scanning /content/drive/MyDrive/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 186090.4it/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.9it/s 4.3s
                   all        128        929      0.685      0.653      0.697      0.528
                person         61        254      0.785       0.69      0.784      0.565
               bicycle          3          6      0.929      0.333      0.392      0.345
                   car         12         46      0.574      0.239      0.306      0.191
            motorcycle          4          5      0.638        0.8      0.906      0.746
              airplane          5 

## 7 Testar o modelo em uma imagem do dataset

In [29]:
# Testar o modelo em uma imagem do dataset
test_image = f"{base_path}/images/train2017/000000000625.jpg"  # trocar imagem aqui
results = model.predict(source=test_image, save=True, imgsz=640, project=runs_path, name="predictions")

print("✅ Treinamento finalizado!")
print(f"📂 Modelos e resultados salvos em: {runs_path}")



image 1/1 /content/drive/MyDrive/coco128/images/train2017/000000000625.jpg: 448x640 3 persons, 1 frisbee, 44.5ms
Speed: 1.2ms preprocess, 44.5ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to /content/drive/MyDrive/coco128/runs/predictions
✅ Treinamento finalizado!
📂 Modelos e resultados salvos em: /content/drive/MyDrive/coco128/runs
